In [38]:
#Conexión Snowflake / Spark 
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

load_dotenv(".env", override=True)

spark = (
    SparkSession.builder
    .appName("Deber03_Analisis_Analytics")
    .config("spark.sql.shuffle.partitions", "4")
    .getOrCreate()
)

SF_OPTIONS = {
    "sfURL": f"{os.getenv('SNOWFLAKE_ACCOUNT')}.snowflakecomputing.com",
    "sfDatabase": os.getenv("SNOWFLAKE_DATABASE"),
    "sfWarehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "sfUser": os.getenv("SNOWFLAKE_USER"),
    "sfPassword": os.getenv("SNOWFLAKE_PASSWORD"),
    "sfSchema": "ANALYTICS",
}

print("Sesión Spark y conexión Snowflake listas.")


Sesión Spark y conexión Snowflake listas.


In [39]:
# Lectura de TRIPS_ENRICHED 
trips = (
    spark.read.format("snowflake")
    .options(**SF_OPTIONS)
    .option("dbtable", "TRIPS_ENRICHED")
    .load()
)

print(f"Tabla ANALYTICS.TRIPS_ENRICHED cargada ({trips.count():,} filas)")
trips.createOrReplaceTempView("trips_enriched")
trips.printSchema()


Tabla ANALYTICS.TRIPS_ENRICHED cargada (16,033,686 filas)
root
 |-- VENDORID: decimal(38,0) (nullable = true)
 |-- VENDOR_NAME: string (nullable = true)
 |-- PICKUP_DATETIME: timestamp (nullable = true)
 |-- DROPOFF_DATETIME: timestamp (nullable = true)
 |-- PASSENGER_COUNT: double (nullable = true)
 |-- TRIP_DISTANCE: double (nullable = true)
 |-- RATECODEID: double (nullable = true)
 |-- RATE_CODE_DESC: string (nullable = true)
 |-- STORE_AND_FWD_FLAG: string (nullable = true)
 |-- PULOCATIONID: decimal(38,0) (nullable = true)
 |-- DOLOCATIONID: decimal(38,0) (nullable = true)
 |-- PAYMENT_TYPE: double (nullable = true)
 |-- PAYMENT_TYPE_DESC: string (nullable = true)
 |-- FARE_AMOUNT: double (nullable = true)
 |-- EXTRA: double (nullable = true)
 |-- MTA_TAX: double (nullable = true)
 |-- TIP_AMOUNT: double (nullable = true)
 |-- TOLLS_AMOUNT: double (nullable = true)
 |-- IMPROVEMENT_SURCHARGE: double (nullable = true)
 |-- TOTAL_AMOUNT: double (nullable = true)
 |-- CONGESTION_SUR

In [40]:
# Verificación rápida 
spark.sql("""
SELECT service_type,
       COUNT(*) AS total_viajes,
       ROUND(AVG(trip_distance),2) AS avg_distancia,
       ROUND(AVG(total_amount),2) AS avg_total
FROM trips_enriched
GROUP BY service_type
""").show()


+------------+------------+-------------+---------+
|service_type|total_viajes|avg_distancia|avg_total|
+------------+------------+-------------+---------+
|      yellow|    14745987|         2.87|    17.19|
|       green|     1287699|         3.85|    18.36|
+------------+------------+-------------+---------+



In [41]:
#a) ¿Cuántos viajes hay por tipo de servicio (service_type)?

spark.sql("""
SELECT service_type, COUNT(*) AS total_viajes
FROM trips_enriched
GROUP BY service_type
ORDER BY total_viajes DESC
""").show()


+------------+------------+
|service_type|total_viajes|
+------------+------------+
|      yellow|    14745987|
|       green|     1287699|
+------------+------------+



In [42]:
#b) ¿Cuál es el promedio de distancia (trip_distance) por tipo de pago?

spark.sql("""
SELECT payment_type_desc AS tipo_pago,
       ROUND(AVG(trip_distance),2) AS distancia_promedio
FROM trips_enriched
GROUP BY payment_type_desc
ORDER BY distancia_promedio DESC
""").show()


+-----------+------------------+
|  tipo_pago|distancia_promedio|
+-----------+------------------+
|       NULL|              9.91|
|    Unknown|              4.01|
|Credit card|              3.04|
|       Cash|              2.51|
|    Dispute|              2.45|
|  No charge|              2.26|
+-----------+------------------+



In [43]:
#c) ¿Qué zona de origen (pu_zone) tiene más viajes?

spark.sql("""
SELECT pu_zone AS zona_origen,
       COUNT(*) AS total_viajes
FROM trips_enriched
WHERE pu_zone IS NOT NULL
GROUP BY pu_zone
ORDER BY total_viajes DESC
LIMIT 10
""").show(truncate=False)


+----------------------------+------------+
|zona_origen                 |total_viajes|
+----------------------------+------------+
|Upper East Side South       |627677      |
|Upper East Side North       |606724      |
|Midtown Center              |600067      |
|Midtown East                |532233      |
|Penn Station/Madison Sq West|508377      |
|Times Sq/Theatre District   |504609      |
|Clinton East                |465534      |
|Union Sq                    |460419      |
|Murray Hill                 |460162      |
|Lincoln Square East         |451966      |
+----------------------------+------------+



In [45]:
#d) ¿Qué zona de destino (do_zone) tiene más viajes?
spark.sql("""
SELECT do_zone AS zona_destino,
       COUNT(*) AS total_viajes
FROM trips_enriched
WHERE do_zone IS NOT NULL
GROUP BY do_zone
ORDER BY total_viajes DESC
LIMIT 10
""").show(truncate=False)


+-------------------------+------------+
|zona_destino             |total_viajes|
+-------------------------+------------+
|Upper East Side North    |646117      |
|Upper East Side South    |568181      |
|Midtown Center           |564525      |
|Murray Hill              |471764      |
|Midtown East             |449345      |
|Times Sq/Theatre District|434177      |
|Lincoln Square East      |414712      |
|Clinton East             |407774      |
|Union Sq                 |398974      |
|Upper West Side South    |395606      |
+-------------------------+------------+



In [46]:
#e) Promedio de monto total (total_amount) por zona de destino
spark.sql("""
SELECT do_zone,
       ROUND(AVG(total_amount),2) AS promedio_monto
FROM trips_enriched
WHERE do_zone IS NOT NULL
GROUP BY do_zone
ORDER BY promedio_monto DESC
LIMIT 10
""").show(truncate=False)


+---------------------------------+--------------+
|do_zone                          |promedio_monto|
+---------------------------------+--------------+
|Rossville/Woodrow                |100.11        |
|Newark Airport                   |97.07         |
|Charleston/Tottenville           |95.34         |
|Eltingville/Annadale/Prince's Bay|94.33         |
|Arden Heights                    |90.56         |
|Outside of NYC                   |84.54         |
|Great Kills                      |82.39         |
|Oakwood                          |80.23         |
|Mariners Harbor                  |76.62         |
|New Dorp/Midland Beach           |76.57         |
+---------------------------------+--------------+



In [47]:
#f) ¿Cuántos viajes por mes?
spark.sql("""
SELECT source_year, source_month,
       COUNT(*) AS total_viajes
FROM trips_enriched
GROUP BY source_year, source_month
ORDER BY source_year, source_month
""").show()


+-----------+------------+------------+
|source_year|source_month|total_viajes|
+-----------+------------+------------+
|       2019|           1|     8368722|
|       2019|           2|     7664964|
+-----------+------------+------------+



In [48]:
#g) Promedio de propina (tip_amount) por tipo de servicio
spark.sql("""
SELECT service_type,
       ROUND(AVG(tip_amount),2) AS promedio_propina
FROM trips_enriched
GROUP BY service_type
""").show()


+------------+----------------+
|service_type|promedio_propina|
+------------+----------------+
|      yellow|            1.98|
|       green|            0.82|
+------------+----------------+



In [49]:
#h) ¿Cuál es la tarifa promedio (fare_amount) por tipo de tarifa (rate_code_desc)?

spark.sql("""
SELECT rate_code_desc AS tipo_tarifa,
       ROUND(AVG(fare_amount),2) AS promedio_tarifa
FROM trips_enriched
GROUP BY rate_code_desc
ORDER BY promedio_tarifa DESC
""").show(truncate=False)


+------------------+---------------+
|tipo_tarifa       |promedio_tarifa|
+------------------+---------------+
|Group ride        |91.88          |
|Nassau/Westchester|69.57          |
|Newark            |64.42          |
|JFK               |51.68          |
|NULL              |46.84          |
|Negotiated fare   |35.97          |
|Standard rate     |11.38          |
+------------------+---------------+



In [50]:
#i) Top 10 combinaciones de zonas origen–destino más frecuentes

spark.sql("""
SELECT pu_zone, do_zone, COUNT(*) AS frecuencia
FROM trips_enriched
WHERE pu_zone IS NOT NULL AND do_zone IS NOT NULL
GROUP BY pu_zone, do_zone
ORDER BY frecuencia DESC
LIMIT 10
""").show(truncate=False)


+---------------------+---------------------+----------+
|pu_zone              |do_zone              |frecuencia|
+---------------------+---------------------+----------+
|N/A                  |N/A                  |193576    |
|Upper East Side South|Upper East Side North|94882     |
|Upper East Side North|Upper East Side North|88251     |
|Upper East Side North|Upper East Side South|80845     |
|Upper East Side South|Upper East Side South|74426     |
|Upper West Side South|Upper West Side North|46734     |
|Upper West Side South|Lincoln Square East  |43142     |
|Lincoln Square East  |Upper West Side South|41731     |
|Upper West Side North|Upper West Side South|40421     |
|Lenox Hill West      |Upper East Side North|37924     |
+---------------------+---------------------+----------+



In [51]:
#j) Distribución de pasajeros (passenger_count) por servicio
spark.sql("""
SELECT service_type,
       passenger_count,
       COUNT(*) AS viajes
FROM trips_enriched
GROUP BY service_type, passenger_count
ORDER BY service_type, passenger_count
""").show()

+------------+---------------+--------+
|service_type|passenger_count|  viajes|
+------------+---------------+--------+
|       green|           NULL|   82054|
|       green|            0.0|    2571|
|       green|            1.0| 1039475|
|       green|            2.0|   85174|
|       green|            3.0|   15648|
|       green|            4.0|    5950|
|       green|            5.0|   38251|
|       green|            6.0|   18523|
|       green|            7.0|      18|
|       green|            8.0|      23|
|       green|            9.0|      12|
|      yellow|           NULL|   58335|
|      yellow|            0.0|  236531|
|      yellow|            1.0|10411262|
|      yellow|            2.0| 2157785|
|      yellow|            3.0|  602052|
|      yellow|            4.0|  269559|
|      yellow|            5.0|  627838|
|      yellow|            6.0|  382496|
|      yellow|            7.0|      49|
+------------+---------------+--------+
only showing top 20 rows



In [52]:
#k) Promedio del total (total_amount) por cantidad de pasajeros

spark.sql("""
SELECT passenger_count,
       ROUND(AVG(total_amount),2) AS promedio_total
FROM trips_enriched
GROUP BY passenger_count
ORDER BY passenger_count
""").show()


+---------------+--------------+
|passenger_count|promedio_total|
+---------------+--------------+
|           NULL|         48.25|
|            0.0|         18.88|
|            1.0|         16.95|
|            2.0|         17.19|
|            3.0|         16.98|
|            4.0|         16.98|
|            5.0|         16.86|
|            6.0|         16.69|
|            7.0|         43.58|
|            8.0|         44.84|
|            9.0|         36.66|
+---------------+--------------+



In [53]:
#l) Promedio de distancia por año y mes

spark.sql("""
SELECT source_year, source_month,
       ROUND(AVG(trip_distance),2) AS avg_distancia
FROM trips_enriched
GROUP BY source_year, source_month
ORDER BY source_year, source_month
""").show()


+-----------+------------+-------------+
|source_year|source_month|avg_distancia|
+-----------+------------+-------------+
|       2019|           1|         2.91|
|       2019|           2|          3.0|
+-----------+------------+-------------+



In [54]:
#m) Monto promedio por tipo de pago y tipo de servicio

spark.sql("""
SELECT service_type, payment_type_desc,
       ROUND(AVG(total_amount),2) AS monto_promedio
FROM trips_enriched
GROUP BY service_type, payment_type_desc
ORDER BY monto_promedio DESC
""").show()


+------------+-----------------+--------------+
|service_type|payment_type_desc|monto_promedio|
+------------+-----------------+--------------+
|      yellow|             NULL|         50.04|
|       green|             NULL|         46.98|
|      yellow|        No charge|         30.76|
|       green|      Credit card|         19.55|
|      yellow|      Credit card|         18.17|
|       green|          Unknown|         14.06|
|      yellow|             Cash|         13.83|
|       green|             Cash|         11.55|
|      yellow|          Dispute|          8.25|
|       green|          Dispute|          2.86|
|       green|        No charge|          1.23|
+------------+-----------------+--------------+



In [55]:
#n) Viajes con airport_fee > 0

spark.sql("""
SELECT COUNT(*) AS viajes_con_airport_fee,
       ROUND(AVG(airport_fee),2) AS promedio_fee
FROM trips_enriched
WHERE airport_fee > 0
""").show()


+----------------------+------------+
|viajes_con_airport_fee|promedio_fee|
+----------------------+------------+
|                     0|        NULL|
+----------------------+------------+



In [56]:
#o) Promedio de total_amount por borough de origen

spark.sql("""
SELECT pu_borough,
       ROUND(AVG(total_amount),2) AS promedio_total
FROM trips_enriched
WHERE pu_borough IS NOT NULL
GROUP BY pu_borough
ORDER BY promedio_total DESC
""").show()


+-------------+--------------+
|   pu_borough|promedio_total|
+-------------+--------------+
|          EWR|         91.59|
|          N/A|         69.42|
|Staten Island|          56.2|
|       Queens|         37.84|
|        Bronx|         26.88|
|     Brooklyn|         21.34|
|      Unknown|         18.32|
|    Manhattan|         15.04|
+-------------+--------------+



In [57]:
#p) Propina promedio por borough de destino

spark.sql("""
SELECT do_borough,
       ROUND(AVG(tip_amount),2) AS promedio_propina
FROM trips_enriched
WHERE do_borough IS NOT NULL
GROUP BY do_borough
ORDER BY promedio_propina DESC
""").show()


+-------------+----------------+
|   do_borough|promedio_propina|
+-------------+----------------+
|          EWR|           11.92|
|          N/A|            8.62|
|Staten Island|            3.81|
|       Queens|            2.73|
|     Brooklyn|            2.29|
|    Manhattan|            1.78|
|      Unknown|            1.65|
|        Bronx|            0.82|
+-------------+----------------+



In [58]:
#q) Número de viajes con tolls_amount > 0

spark.sql("""
SELECT COUNT(*) AS viajes_con_peaje,
       ROUND(AVG(tolls_amount),2) AS promedio_peaje
FROM trips_enriched
WHERE tolls_amount > 0
""").show()


+----------------+--------------+
|viajes_con_peaje|promedio_peaje|
+----------------+--------------+
|          868518|          6.15|
+----------------+--------------+



In [59]:
#r) Promedio de tiempo entre pickup y dropoff

spark.sql("""
SELECT ROUND(AVG(
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 60
), 2) AS minutos_promedio
FROM trips_enriched
""").show()


+----------------+
|minutos_promedio|
+----------------+
|           17.35|
+----------------+



In [60]:
#s) Viajes con total_amount negativo o cero (datos anómalos)

spark.sql("""
SELECT COUNT(*) AS viajes_anomalos
FROM trips_enriched
WHERE total_amount <= 0
""").show()


+---------------+
|viajes_anomalos|
+---------------+
|          27198|
+---------------+



In [61]:
#t) Propina promedio por método de pago

spark.sql("""
SELECT payment_type_desc,
       ROUND(AVG(tip_amount),2) AS promedio_propina
FROM trips_enriched
GROUP BY payment_type_desc
ORDER BY promedio_propina DESC
""").show()


+-----------------+----------------+
|payment_type_desc|promedio_propina|
+-----------------+----------------+
|      Credit card|            2.65|
|             NULL|            0.03|
|        No charge|             0.0|
|          Dispute|             0.0|
|             Cash|             0.0|
|          Unknown|             0.0|
+-----------------+----------------+

